<p style="text-align: center;font-size: 40pt">Use case 1 - Search and Rescue</p>

# Overview 

Requirements
- [Error minimization](4-lesson_error_minimization.ipynb)

Objectives of this lesson:
- give an example of application for point cloud registration

# Introduction

This first use case look into a registration solution for the robot presented in the [lesson Example of autonomous vehicles](../autonomous_vehicles/2-lesson_examples.ipynb).
Within the framework of the European project NIFTi (FP7-ICT-247870), novel solutions were assembled together and tested in order to support firemen and first responders in Search \& Rescue missions.
A first use of 3D maps is to help plan the of strategic deployment of responders in environment were humans can only intervene for a limited time.
Those situations include nuclear incident, chemical spill, unstable supporting structures, and excessive heat conditions.
When tele-operated, 3D maps can be used to provide the user with situation awareness and support critical decisions about risky platform motions.
This type of application often has a limited communication range, leading to an increased need for autonomous behavior such as navigation [[Colas et al., 2013]](https://ieeexplore.ieee.org/document/6696431).
More autonomy also means more onboard computation in case of communication breakdown.
In such situations, onboard localization is essential to return the platform to a location were wireless communication can be reestablished.

Apart from increasing pressure on real-time solutions, Search \& Rescue environments cover a large spectrum of possibilities.
For example, deployments can happen in a well-structured nuclear plant, in a partially-collapsed building or outdoors in a case of a train chemical spill.
In an advanced robot-human collaborative intervention, dynamic elements created by the other agents (e.g., firemen, other robots, etc.) acting on the field need to be considered in the registration solution.
Moreover, dynamic elements at a global level (e.g., building collapsing during the exploration) can happen.
In the case of relatively contained situations, where humans have difficulty accessing a restricted intervention area, very little dynamic elements are expected (e.g., the Fukishima Daiichi nuclear disaster in 2011).
In the cases presented in this section, applications were demonstrated with few dynamic elements and without the need to identify them.

# Platform
The platform deployed in the field is called NiftiBot ([Figure 3.1](#robotNifti)).
Its mobility is provided by two tracked bogies linked by a differential suspension to facilitate the crossing of uneven terrain.
Moreover, two flippers per track allow an active control of the platform orientation and offer an extended range for gap traversals.
The mechanical configuration of the robot enables it to climb slopes up to 45$^\circ$, including stairs.
The robot fits in a bounding box of 0.17\,m$^3$ volume and weights approximately 20\,kg.
The primary sensor used for registration was a 2D rotating SICK LMS-151 laser, with its rotation axis pointing toward the front of the robot.
The aggregation of 2D scans used the motion information of the platform to reconstruct 3D scan at 0.35\,Hz.
A typical 3D scan contained 55,000 points.
Two other sensors can be found on the platform, namely a GPS-aided IMU (X-sens MTI-G ) and an omnidirectional camera (PointGrey Ladybug2).
The velocity of the platform can be considered slow (0.3 m/s), especially during teleoperation where delicate motion are required.
The pre-alignment information used as input for the registration was based on a Kalman filter fusing the IMU and the odometry information.
The large error on motion estimates came from the vibration of the tracks, the large contact surface of the tracks on the ground, and the fact that the platform often collides with obstacles.
Therefore, smooth and continuous motion models can easily break, thus simple prediction models (e.g., constant velocity) are not applicable.
<p id="robotNifti" style="text-align: center;">
    <img src="images/robot_nifti.jpg" width="50%"/> <br/>
    <b>Figure 3.1:</b> Photograph of NiftiBot, the main platform used for Search \& Rescue demonstrations in an unstructured environment. 
    The 3D sensor, Sick LMS-151, is positioned in front of the robot, in-between the two white tracks. 
    The extra motorized rotation axis is pointing in the forward direction of the robot.
</p>

# Solution
The configuration of the rotating laser produces a high density of points in front of the robot, which is desirable to predict collision, but not beneficial to the registration minimization. 
Thus, we forced the maximal density to 100 points per m$^3$ after having randomly subsampled the point cloud in order to finish the registration and the map maintenance within 2 seconds.
We expected the error on pre-alignment of the 3D scans to be less than 0.5\,m based on the velocity of the platform and the number of time ICP would be computed per second. 
We used this value to limit the matching distance.
We also removed paired points with an angle difference larger than 50$^\circ$ to avoid, for example, points from different side of a wall to match together.
This happen often when the robot is moving through different rooms.
As for the global map, we maintained a density of 100 points per m$^3$ every time a new input scan was merged in it.
A maximum of 600,000 points were kept in memory to avoid degradation of the computation time performance when exploring a larger environment than expected.
The complete list of modules used with their main parameters can be found in [Table 3.1](#icpConfigAppNifti).
<p id="icpConfigAppNifti" style="text-align: center;margin-left: 15%;margin-right: 15%;">
    <b>Table 3.1:</b> Configuration of the ICP chain for the Search & Rescue mapping applications. 
    The definition of the column <em>Step</em> follows previous lessons. 
    The names used in the column <em>Module</em> refer to specific implementation documented in the open source library <tt>libpointmatcher</tt>.
</p>


|    *Step*   |            *Module*            | *Description* |
|:-----------:|:------------------------------:|:-------------:|
| $DF_{read}$ |   <tt>SimpleSensorNoise</tt>   | Add uncertainty for the SICK LMS sensor as observed in [[Pomerleau et al., 2012a]](https://ieeexplore.ieee.org/document/6473358). |
|             | <tt>SamplingSurfaceNormal</tt> | Keep 80 % of the points, while extracting surface normals based on 20 NN. |
|             |  <tt>ObservationDirection</tt> | Add vector pointing toward the origin of the sensor. |
|             |     <tt>OrientNormals</tt>     | Orient surface normals toward the observation direction. |
|             |       <tt>MaxDensity</tt>      | Subsample to keep point with density of 100 pts/m$^3$. |
|  $DF_{ref}$ |     <tt>SurfaceNormal</tt>     | Compute normal and density with  20 NN and an approximation factor $\epsilon=3.16$. |
|             |       <tt>MaxDensity</tt>      | Subsample to keep point with density of 100 pts/m$^3$. |
|             |     <tt>MaxPointCount</tt>     | Subsample 70 % if there is more than 600,000 points. |
|      MF     |         <tt>KDTree</tt>        | Use an approximate kD-tree with a maximum matching distance of 0.5 m and an approximation factor of $\epsilon=3.16$. |
|      OF     |      <tt>TrimmedDist</tt>      | Keep 80 % closest paired points. |
|             |     <tt>SurfaceNormal</tt>     | Remove paired points with normals angle larger than 50$^\circ$. |
|      EM     |      <tt>PointToPlane</tt>     | Objective function using point-to-plane error. |
|      TC     |      <tt>Differential</tt>     | Stop after a minimum error below 0.01 m and 0.001 rad. |
|             |        <tt>Counter</tt>        | Stop after the iteration count reached 40. |
|             |         <tt>Bound</tt>         | Stop if transformation increases beyond 5.0 m and 0.8 rad. |
<p style="text-align: center;margin-left: 15%;margin-right: 15%;font-size:10px;">
    <em>Legend</em>: $DF_{read}$ = Data Filters for readings, $DF_{ref}$ = Data Filters for references, MF = Matching Function, OF = Outlier Filters, EM = Error Minimizer, TC = Transformation Checker.
</p>

# Indoor Preliminary Tests

To test the range of the platform motions and to demonstrate the need for 3D reconstructions, we ran an experiment in our laboratory leading to the reconstruction of a full staircase ([Figure 3.2](#7floorMap)).
The robot started its path in an office located on the E-floor, and was driven down a staircase two floors below (C-floor, in the basement).
The robot was controlled using a joystick by an operator following it throughout the path.
The robot was then driven six floors up to the I-floor using the same staircase.
In this application, the robot acquired scans from a stop-and-go strategy with a scan taken roughly every 2\,m.
<p id="7floorMap" style="text-align: center;">
    <img src="images/7_floors.png" width="75%"/> <br/>
    <b>Figure 3.2:</b> Mapping of a 7-floor staircase using a Search \& Rescue robot.
    <em>Right</em>: Side view of the resulting map with the floors colored based on elevation.
    <em>Left</em>: Top view of the E-floor with the ceiling removed and the points colored based on elevation, red being higher, blue lower.
</p>

The complete map was processed onboard the robot.
Because the information of the past environment was fused in the global map while the robot went down the stairs, the drift in the localization was considerably reduced on the way up.
This experiment comprised two critical moments: (1) when the robot moved out of the office and (2) when the robot entered the basement (C-floor).
In both situations the overlap between the new scan and the global map was minimal.
This information was known by the operator, so more scans were taken at those moments to avoid large deviations in the global map.

# Rail Yard
On two occasions, the NIFTi platform was tested outdoors in a rail yard, with the permission of the <em>Depo kolejových vozidel Praha</em> (Prague Depot of Rail Vehicles, Czech Republic).
In the first experiment, the robot was also driven in the yard by operators who were following the platform.
The 3D reconstruction is shown in [Figure 3.3](#nifti_railyard_1).
The robot started its journey at one corner of a railcar, going along the railcar flank to the other corner and then, turned back to the starting position passing through the vegetation located on the other side of the railcar.
Even if the path contained a loop, the precision of the registration was accurate enough to properly match the first scan with the ones recorded at the end.
<p id="nifti_railyard_1" style="text-align: center;">
    <img src="images/nifti_railyard_1_side.png" width="60%"/>
    <img src="images/nifti_railyard_1_top.png" width="60%"/> <br/>
    <b>Figure 3.3:</b> Deployment of the NiftiBot in a rail yard with a single railcar and dense vegetation.
    <em>Top</em>: Side view of the reconstructed environment with the railcar in the middle and the vegetation behind.
    <em>Bottom</em>: Top view of the reconstructed environment.
    For both images, colors of the point clouds were chosen to ease the comprehension of the 3D-scene.
</p>

In the second experiment, the robot was driven to explore inside an old railcar where a person was standing still in the shadow to test in parallel the capability of the thermal camera.
The robot then went out of the railcar, crossed dense vegetation, followed the side of a more modern railcar and stopped in front of it, where a second operator was captured in the global map.
<p id="nifti_railyard_2" style="text-align: center;">
    <img src="images/nifti_railyard_2_side_text.png" width="50%"/> <br/>
    <b>Figure 3.4:</b> Bird's-eye view of the second experiment where the robot explored inside an old wooden railcar, crossed some dense vegetation and finished its path around a more modern railcar.
</p>

For both experiments, the robot acquired scans with a stop-and-go strategy.
The scans were gathered at uneven distances (up to 8\,m apart) by operators without prior knowledge about critical situations.
All the 3D scans were processed offline four times faster than the speed at which they were recorded.
Those experiments are a good example of semi-structured environments, where trees and dense vegetation share the scene with planar surfaces from trains.

# Collapsed Church
In May 2012 a sequence of earthquakes hit the Emilia-Romagna region, Northern Italy, with a magnitude of 5.8.
Three month later, NIFTi partners deployed the platform with the support of the <em>Vigili del Fuoco</em> (National Fire-watchers' Corps of Italy) and the <em>Beni Culturali</em> (Ministry of Culture of Italy) in Mirandola for a damage assessment mission. 
One of the visited sites was the <em>Chiesa di San Francesco d'Assisi</em>, in which 3D scans were recorded.
The robot started outside the church, crossed a door and realized a straight line, navigating on the cluttered floor of the western gallery of the church ([Figure 3.5](#nifti_church)).
One can observe on the reconstruction the pillars and arches supporting the remaining roof of the church.
The level of damage of the church was quite important, thus limiting the exploration possibilities of the platform, as depicted in [Figure 3.6](#nifti_church_photo).
<p id="nifti_church" style="text-align: center;">
    <img src="images/nifti_san_francesco_side.png" width="60%"/>
    <img src="images/nifti_san_francesco_top.png" width="60%"/> <br/>
    <b>Figure 3.5:</b> Reconstruction of the <em>Chiesa di San Francesco d'Assisi</em> with the color following elevation.
    <em>Top</em>: Side view of the reconstruction.
    <em>Bottom</em>: Top view of the church.
</p>

<p id="nifti_church_photo" style="text-align: center;">
    <img src="images/nifti_san_francesco_photo.jpg" width="36.75%"/>
    <img src="images/nifti_san_francesco_inside.png" width="36.75%"/> <br/>
    <b>Figure 3.6:</b> Comparison of the point cloud reconstruction with a photograph taken during exploration.
    <em>Left</em>: Photograph of the western gallery with the collapse roof on the right.
    <em>Right</em>: Front view of the reconstruction.
</p>

The platform was remotely operated from a control station situated outside the church and was able to continuously scan the environment while moving in the environment.
Again, all the 3D scans were processed offline four times faster than the speed at which they were recorded.
It is another example of semi-structured environment but, in this case, the unstructured part comes from rubble following the partial collapse of the church. 

# Collaborative Mapping

Within the framework of the European project <em>sFly</em> (FP7-ICT-231855), three micro-helicopters (AscTec Firefly) were deployed over a Search \& Rescue training site in Zurich, Switzerland.
The platform used is shown in [Figure 3.7](#robot_sfly) flying over a collapsed concrete building.
The three Fireflies were sent so that each one covered a pre-determined part of the environment and streamed back images to a control station.
The collected images were used by the ETH Computer Vision and Geometry Group to reconstruct a 3D representation of the environment explored ([Figure 3.8](#resultsZurich) - <em>Bottom left</em>).
<p id="robot_sfly" style="text-align: center;">
    <img src="images/robot_sfly.jpg" width="60%"/> <br/>
    <b>Figure 3.7:</b> Photograph of one of the three AscTec Fireflies used to map the environment in collaboration with the NiftiBot.
</p>

Another map was generated using a ground platform (i.e., NiftiBot).
The robot was tele-operated on a road around the main collapsed building presented in [Figure 3.8](#resultsZurich), for a path totaling 110 m long.
The operator had a good prior knowledge of the environment before driving the robot around from a control station.
The large road coupled with the awareness of the environment contributed to increase the velocity of the robot while exploring the area.
The resulting map of the ground robot and the map of the Fireflies were then fused using a standalone ICP implementation taken from our registration library <tt>libpointmatcher</tt>.
Both maps were having roughly 300,000 points and were registered using the configuration of [Table 3.2](#icpConfigAppcallaborative).
The final map is depicted at the bottom right of [Figure 3.8](#resultsZurich).
<p id="icpConfigAppcallaborative" style="text-align: center;margin-left: 15%;margin-right: 15%;">
    <b>Table 3.2:</b> Configuration of the ICP chain for the collaborative mapping applications.
    The definition of the column <em>Step</em> follows previous lessons. 
    The names used in the column <em>Module</em> refer to specific implementation documented in the open source library <tt>libpointmatcher</tt>.
</p>


|    *Step*   |            *Module*            | *Description* |
|:-----------:|------------------------------|:-------------:|
| $$DF_{read}$$ |     <tt>RandomSampling</tt>    | Keep randomly 15 % of the points. |
|  $$DF_{ref}$$ | <tt>SamplingSurfaceNormal</tt> | Keep 15 % of the points, while extracting surface normals based on 50 NN. |
|      MF     |         <tt>KDTree</tt>        | Use a kD-tree with a maximum matching distance of 1.0 m and match a point from the reading to 5 others in the reference. |
|      OF     |      <tt>TrimmedDist</tt>      | Keep 80 % closest points. |
|      EM     |      <tt>PointToPlane</tt>     | Objective function using point-to-plane error. |
|      TC     |      <tt>Differential</tt>     | Stop after a minimum error below 0.01 m and 0.001 rad. |
|             |        <tt>Counter</tt>        | Stop after the iteration count reached 100. |


<p style="text-align: center;margin-left: 15%;margin-right: 15%;font-size:10px;">
    <em>Legend</em>: $DF_{read}$ = Data Filters for readings, $DF_{ref}$ = Data Filters for references, MF = Matching Function, OF = Outlier Filters, EM = Error Minimizer, TC = Transformation Checker.
</p>

<p id="resultsZurich" style="text-align: center;">
    <img src="images/zurich_trainingSite.jpg" width="36.75%"/>
    <img src="images/zurich_nifti_new.png" width="36.75%"/>
    <img src="images/zurich_vision_new_text.png" width="36.75%"/>
    <img src="images/zurich_combined_new.png" width="36.75%"/> <br/>
    <b>Figure 3.8:</b> Resulting maps of the Zurich firefighter training site.
    <em>Top right</em>: Photograph of the training site with a partially collapsed tower in the middle.
    <em>Top left</em>: Top view of the reconstruction realized with the data from the ground robot with the same tower in the middle of the map.
    <em>Bottom left</em>: Top view of the reconstruction realized with the data from the three Fireflies.
    <em>Bottom right</em>: Top view of the combined map.
    Note that the color correspond the elevation: light gray is low, dark blue is high.
</p>

At that time, the Kalman filter used to fused the odometry with the IMU was not well tuned and bias in the estimation induced drift on yaw estimates.
Roughly, a constant drift of 5$^\circ$/s was estimated visually.
The scans were gathered while the ground platform was moving, which generated a larger localization error than prior experiments.
A total of four runs were recorded with the ground platform: 
1. continuous scanning, turning clockwise around the main building; 
2. continuous scanning, counterclockwise; 
3. stop-and-go scanning, clockwise; and 
4. stop-and-go scanning, counterclockwise.

Two experiments out of four closed the loop with a negligible error at the closing point.
Surprisingly, the successful runs were the ones turning counterclockwise, contradicting our first intuition that stop-and-go scanning would be more accurate.
This experiment highlighted the importance of correcting IMU drift with an external registration algorithm and also showed that the robot could scan while moving.
We selected the resulting representation of the second run to fuse both sources of information (i.e., laser and camera) in a common 3D reconstruction.

# Conclusion
You should do the following activities to enhance your understanding of the concepts viewed in this lesson:
- modify the markdown by adding your own notes using `> my notes`; and
- complete the tables [Symbol definitions](#Symbol-definitions) and [Glossary](#Glossary) and add your own definitions.

Parallel lessons:
- [Use case 2](5-lesson_use_case_2.ipynb)
- [Use case 3](5-lesson_use_case_3.ipynb)

Next lesson:
- [Registration challenges](6-lesson_challenges.ipynb)

## Symbol definitions

| Symbol             | Definition  |
|--------------------|-------------|
|                    |             |

## Glossary

| English   | Français   | Definition |
|-----------|------------|------------|
|           |            |            |